In [312]:
import pandas as pd
import numpy as np

In [313]:
pageURL  = 'https://it.wikipedia.org/wiki/Leone_d%27oro_al_miglior_film'
tables = pd.read_html(pageURL, match='Anno', header=0)
dataframe = tables[0]

In [314]:
dataframe['index_col'] = dataframe.index

In [315]:
dataframe[14:15]

,Anno,Film,Regista,Nazione,index_col
14,Il generale Della Rovere,Roberto Rossellini,Italia,NaN,14


In [316]:
# creo un dataframe per i record con le colonne mappate male, in cui l'anno contiene stringhe
df2=dataframe[dataframe['Anno'].str.contains("[a-zA-Z]")]
# creo un dataframe per tutti gli altri valori
df1=dataframe[(~dataframe['Anno'].str.contains("[a-zA-Z]")) & (dataframe['Regista'].str.contains(".+")) ]

In [317]:
# aggiungo un colonna con valori nulli e la sposto a inizio dataframe
df2.loc[:,'test']=np.nan
col = "test" 
df2 = pd.concat([df2[col],df2.drop(col,axis=1)], axis=1)

In [318]:
# rimuovo da df2 la colonna in più
colonna=len(df2.columns)-2
df2.drop(df2.columns[colonna], axis = 1,inplace=True)

In [319]:
# rinomino le colonne dei dataframe con dei numeri crescenti da 0 a salire
df2.columns = np.arange(len(df2.columns))
df1.columns = np.arange(len(df1.columns))

In [320]:
# faccio il merge dei dataframe
df=df1.append(df2, ignore_index=True)

In [321]:
# riordino le righe
df.sort_values([4],inplace=True)
# inserisco l'anno nelle celle in cui manca
df.fillna(method="ffill",inplace=True)
# imposto un separatore per le celle con più nazioni
df.replace({'\/.{1}': ':'}, regex=True,inplace=True)
# rinomino le colonne
df.rename(index=str, columns={0: "Anno", 1: "Titolo",2:"Regista",3:"Nazione"}, inplace=True)
# rimuovo una colonna non utile
df.drop([4], axis=1)
# salvo la lista in CSV
df.to_csv("leoni_pd.csv",index=False,encoding="utf-8")

In [322]:
nazioni=df[['Nazione']]

In [323]:
# splitto le celle con più nazioni associate, traspongo in verticale e creo un dataframe
nazioni=nazioni['Nazione'].str.split(':', expand=True).stack()
nazioni=nazioni.to_frame()

In [324]:
# raggruppo, conteggio e ordino
nazioni=nazioni.groupby([0]).size().reset_index(name='Conteggio').sort_values(['Conteggio'], ascending=False)
nazioni.rename(index=str, columns={0: "Nazione"}, inplace=True)

In [325]:
# salvo in CSV
nazioni.to_csv("listaNazioni_pd.csv",index=False,encoding="utf-8")